# Advanced Machine Learning - Project 1

## TODO
- renaming vars
- nan imputation 
- check how to pre-process features (How to encode categorical data?, How to scale continuous?)
- read into hybrid Bayesian Networks? How to do?
- train test split, do KFold?

## Learning approach
- (maybe feature selection)
1. learn structure
2. learn CPDs

## Make predictions

## Interpret

In [1]:
# Imports

# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

# Data Upscaling or Downscaling
from sklearn.model_selection import train_test_split
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling

# Bayesian network and estimators
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

from pgmpy.estimators import ParameterEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator

from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch



## 1. Data import and cleaning

In [ ]:
# Reading the dataset
df = pd.read_csv("../CHD_dataset.csv")

df.head(20)

: 

In [ ]:
# Rename variables according to this convention
# cat_{varname}: categorical variable
# ord_{varname}: ordinal variable
# cont_{varname}: continuous variable

df.rename(columns={
    'Sex': 'cat_assigned_sex',
    'age': 'cont_age',
    'education': 'ord_education',
    'currentSmoker': 'cat_current_smoker',
    'cigsPerDay': 'cont_cigs_per_day',
    'BPMeds': 'cat_bp-medication',
    'prevalentStroke': 'cat_prev_stroke',
    'prevalentHyp': 'cat_prev_hypertense',
    'diabetes': 'cat_diabetes',
    'totCol': 'cont_total_colesterol',
    'sysBP': 'cont_systolic_bp',
    'diaBP': 'cont_diabolic_bp'

}, inplace=True)

In [10]:
print( df["BPMeds"].min())
print( df["BPMeds"].max())
print( df["BPMeds"].mean())

print( df["totChol"].min())
print( df["totChol"].max())
print( df["totChol"].mean())

print( df["glucose"].min())
print( df["glucose"].max())
print( df["glucose"].mean())
print( df["glucose"].std())

0.0
1.0
0.029615476474802963
107.0
696.0
236.69952267303103
40.0
394.0
81.96365524402907
23.954334811344744


In [ ]:
# NaN imputation

# Checking for NaN
# print(df.isnull().sum())

# Deleting rows with multiple NaN values (in this case, it's none)
df.dropna(thresh = 2, inplace=True)

# Get the mean of cigs per day of smokers
mean_cigs_per_day = df[ (df["currentSmoker"] == "Yes") & ( df["cigsPerDay"] > 0 ) ]["cigsPerDay"].mean(skipna=True)

# Fill the NaN values of smokers with the mean, of non smokers with 0
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "Yes") , "cigsPerDay" ] =  mean_cigs_per_day
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "No"), "cigsPerDay" ] = 0

# Print just to check if operation above was completed correctly
# print(f'NaN in the dataset:\n {df.isnull().sum()["cigsPerDay"]}')

"""
TODO: Fill education
"""
# Get mean BMI for male and female
mean_bmi_male = df.loc[ df["Sex"] == "male", "BMI"].mean()
mean_bmi_female = df.loc[ df["Sex"] == "female", "BMI"].mean()

# Fill NaN values
df.loc[ ( df["Sex"] == "male" ) & ( df["BMI"].isna() ), "BMI"] = mean_bmi_male
df.loc[ ( df["Sex"] == "female" ) & ( df["BMI"].isna() ), "BMI"] = mean_bmi_female


# Get mean BPMeds
mean_bpmeds_male = df.loc[ (df["BPMeds"].notna()) & ( df["Sex"] == "male"), "BPMeds"].mean()
mean_bpmeds_female = df.loc[ (df["BPMeds"].notna()) & (df["Sex"] == "female"), "BPMeds"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["BPMeds"].isna() ), "BPMeds"] = mean_bpmeds_male
df.loc[ (df["Sex"] == "female") & ( df["BPMeds"].isna() ), "BPMeds"] = mean_bpmeds_female


# Get mean totChol
mean_totChol_male = df.loc[ (df["totChol"].notna()) & (df["Sex"] == "male"), "totChol"].mean()
mean_totChol_female = df.loc[ (df["totChol"].notna()) & (df["Sex"] == "female"), "totChol"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["totChol"].isna() ), "totChol"] = mean_totChol_male
df.loc[ (df["Sex"] == "female") & ( df["totChol"].isna() ), "totChol"] = mean_totChol_female


# Get mean glucose
mean_glucose_male = df.loc[ (df["glucose"].notna()) & (df["Sex"] == "male"), "glucose"].mean()
mean_glucose_female = df.loc[ (df["glucose"].notna()) & (df["Sex"] == "female"), "glucose"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["glucose"].isna() ), "glucose"] = mean_glucose_male
df.loc[ (df["Sex"] == "female") & ( df["glucose"].isna() ), "glucose"] = mean_glucose_female


# Get mean heartRate
mean_heartRate_male = df.loc[ (df["heartRate"].notna()) & (df["Sex"] == "male"), "heartRate"].mean()
mean_heartRate_female = df.loc[ (df["heartRate"].notna()) & (df["Sex"] == "female"), "heartRate"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["heartRate"].isna() ), "heartRate"] = mean_heartRate_male
df.loc[ (df["Sex"] == "female") & ( df["heartRate"].isna() ), "heartRate"] = mean_heartRate_female


# Education -> should we do clustering??  TODO: im not sure about using mean in this variable!!
mean_education = df.loc[ df["education"].notna(), "education"].mean()

df.loc[ df["education"].isna(), "education"] = mean_education

# Checking if NaN are corrected 
# print(df.isnull().sum())

In [ ]:
# Convert categorical data to numerical
# Education -> converting to integers (categorical)

# ?!??!?!?! not working for some reason, none of the methods under
#df.loc[ : , "education" ] = df["education"].astype('int32')
# df.loc[:,"education"] = df["education"].apply(lambda x: int(x))
# print(df["education"])


In [ ]:
# Scaling

In [ ]:
# Remove target variable from the dataset
# Split into train and test data (scikit)